**1/ download dataset**

[yahoo finance](https://finance.yahoo.com/quote/MSFT/history/)

**2/ preprocessing data**

**3/ build model**

**4/ train model**

**5/ evaluate model**

**7/ save and load model**

**8/ predict new-input**